In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
from google.colab.patches import cv2_imshow
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
img_path = "/content/drive/MyDrive/UGP_EE492A/PSO_results/images/4.jpg"
img = cv.imread("/content/drive/MyDrive/UGP_EE492A/PSO_results/images/4.jpg")

In [ ]:
img.dtype

In [ ]:
img.shape

In [ ]:
def color_hist(img):

    y = np.linspace(0 ,256)
    fig , ax = plt.subplots(3,1)
    ax[0].hist(img[:,:,0].flatten().ravel(),color='blue',bins = 1846)
    ax[1].hist(img[:,:,1].flatten().ravel(),color='green',bins = 1846)
    ax[2].hist(img[:,:,2].flatten().ravel(),color='red',bins = 1846)

    plt.show()

In [ ]:
def plot_hist(img):

    plt.hist(img.flatten(),bins = 1846)
    plt.show()


In [ ]:
import numpy as np

def suggest_bin_value_for_color_hist(channel):
    total_pixels = np.prod(channel.shape)
    # print(total_pixels)
    suggested_bins = int(np.sqrt(total_pixels))
    return suggested_bins


In [ ]:
suggest_bin_value_for_color_hist(img)

In [ ]:
def image(input):
    val = list(input)

    for p in range(len(val)):
        if val[p][1]=="B":
            b = val[p][0]
        elif val[p][1]=="G":
            g = val[p][0]
        if val[p][1]=="R":
            r = val[p][0]

    img = np.dstack([b,g,r])
    img = np.array(img,dtype=np.uint8)

    return img

In [ ]:
# Indicating superior, inferior and intermediate channels based on mean of pixels in channel
def superior_inferior_split(img):

    B, G, R = cv.split(img)

    pixel = {"B":np.mean(B) ,"G":np.mean(G),"R":np.mean(R)}
    # print("Pixel ", pixel)
    pixel_ordered = dict(sorted(pixel.items(), key=lambda x: x[1], reverse=True))
    # print("pixel_ordered ", pixel_ordered)

    # Classifying Maximum, Minimum and Intermediate channels of image
    label =["Pmax","Pint","Pmin"]
    chanel={}

    for i,j in zip(range(len(label)),pixel_ordered.keys()):
         if j=="B":
             chanel[label[i]]=list([B,j])

         elif j=="G":
             chanel[label[i]]=list([G,j])

         else:
             chanel[label[i]]=list([R,j])
    return chanel

In [ ]:
def neutralize_image(img):

    track = superior_inferior_split(img)

    Pmax = track["Pmax"][0]
    Pint = track["Pint"][0]
    Pmin = track["Pmin"][0]

    #gain_factor Pint
    J = (np.sum(Pmax) - np.sum(Pint))/(np.sum(Pmax) + np.sum(Pint))

    #gain_factor Pmin
    K = (np.sum(Pmax) - np.sum(Pmin))/(np.sum(Pmax) + np.sum(Pmin))

    for i in range(len(track["Pint"][0])):
        for j in range(len(track["Pint"][0][i])):
            if track["Pint"][0][i][j] < 200:
                track["Pint"][0][i][j] += J * Pmax[i][j]

    for i in range(len(track["Pmin"][0])):
        for j in range(len(track["Pmin"][0][i])):
            if track["Pmin"][0][i][j] < 200:
                track["Pmin"][0][i][j] += J * Pmax[i][j]

    #neutralised image
    neu_img = image(track.values())

    return neu_img

In [ ]:
import numpy as np

def Stretching(image, trim_percent=10):
    LSR_img = []  # for lower stretched image
    USR_img = []  # for upper stretched image
    height, width = image.shape[:2]

    for i in range(image.shape[2]):
        img_hist = image[:,:,i]
        max_P = np.max(img_hist)
        min_P = np.min(img_hist)

        # Calculate trimmed mean
        trimmed_mean = np.mean(np.sort(img_hist.flatten())[int(trim_percent/2):-int(trim_percent/2)])

        median_P = np.median(img_hist)
        avg_point = (trimmed_mean + median_P) / 2

        LS_img = np.zeros((height, width))
        US_img = np.zeros((height, width))

        for i in range(0, height):
            for j in range(0, width):
                if img_hist[i][j] < avg_point:
                    LS_img[i][j] = int(((img_hist[i][j] - min_P) * ((255 - min_P) / (avg_point - min_P)) + min_P))
                    US_img[i][j] = 0
                else:
                    LS_img[i][j] = 255
                    US_img[i][j] = int(((img_hist[i][j] - avg_point) * ((255) / (max_P - avg_point))))

        LSR_img.append(LS_img)
        USR_img.append(US_img)

    LS = np.array(np.dstack(LSR_img), dtype=np.uint8)
    US = np.array(np.dstack(USR_img), dtype=np.uint8)

    return LS, US


In [ ]:
def enhanced_image(img1, img2):

    #integration of dual intensity images to get Enhanced-constrast output image
    b1,g1,r1 = cv.split(img1)
    b2,g2,r2 = cv.split(img2)

    height, width = img1.shape[:2]
    dual_img=np.zeros((height, width,3),dtype=np.uint8)

    dual_img[:,:,0] = np.array(np.add(b1/2, b2/2),dtype = np.uint8)
    dual_img[:,:,1] = np.array(np.add(g1/2, g2/2),dtype = np.uint8)
    dual_img[:,:,2] = np.array(np.add(r1/2, r2/2),dtype = np.uint8)

    return dual_img


In [ ]:
#particle class
class Particle:
  def __init__(self, func, dim, vmin, vmax, seed):
    self.rnd = np.random.seed(seed)

    # initialize position, velocity, local_best_particle of the particle with 0.0 value
    self.velocity = np.zeros(dim)
    self.best_part_pos = np.zeros(dim)

    self.position = np.random.uniform(vmin, vmax, dim)

    # compute fitness of particle
    self.fitness = func(self.position) # curr fitness

    # initialize best position and fitness of this particle
    self.best_part_pos = np.copy(self.position)
    self.best_part_fitness = self.fitness     # best fitness


def pso(func, max_iter, num_particles, dim, vmin, vmax, params):

  # hyper parameters
  wmax = params["wmax"]    # maximum inertia
  wmin = params["wmin"]    #minimum inertia
  c1 = params["c1"] 	   # cognitive (particle)
  c2 = params["c2"]       # social (swarm)

  rnd = np.random.seed()

  # create num_particles
  swarm = [Particle(func, dim, vmin, vmax, i) for i in range(num_particles)]

  # compute the value of best_position and best_fitness in swarm
  best_swarm_pos = np.zeros(dim)
  best_swarm_fitness = np.inf # swarm best

  # computer best particle of swarm and it's fitness
  for i in range(num_particles): # check each particle
    if swarm[i].fitness < best_swarm_fitness:
      best_swarm_fitness = swarm[i].fitness
      best_swarm_pos = np.copy(swarm[i].position)

  # main loop of pso
  it = 0
  while it < max_iter:

    # For every 5 iterations print iteration number and best fitness value
    if it % 5 == 0:
      print("Iteration = " + str(it) + " best fitness = %f" % best_swarm_fitness)

    w = wmax - ((wmax - wmin)/max_iter)*it

    for i in range(num_particles):

      # compute new velocity of current particle
      swarm[i].velocity = (
                           (w * swarm[i].velocity) +
                           (c1 * np.random.rand(dim) * (swarm[i].best_part_pos - swarm[i].position)) +
                           (c2 * np.random.rand(dim) * (best_swarm_pos -swarm[i].position))
                         )

      # compute new position using new velocity
      for k in range(dim):
        swarm[i].position[k] += swarm[i].velocity[k]
        swarm[i].position[k] = np.maximum(swarm[i].position[k], vmin)
        swarm[i].position[k] = np.minimum(swarm[i].position[k], vmax)

      # compute fitness of new position
      swarm[i].fitness = func(swarm[i].position)

      # check for local best particle
      if swarm[i].fitness < swarm[i].best_part_fitness:
        swarm[i].best_part_fitness = swarm[i].fitness
        swarm[i].best_part_pos = np.copy(swarm[i].position)

      # check for global best particle
      if swarm[i].fitness < best_swarm_fitness:
        best_swarm_fitness = swarm[i].fitness
        best_swarm_pos = np.copy(swarm[i].position)

    it += 1

  gbest ={}
  gbest["position"] = best_swarm_pos
  gbest["cost"] = best_swarm_fitness

  return gbest

In [ ]:
def pso_image(img):

    group = superior_inferior_split(img)

    maxi = np.mean(group["Pmax"][0])
    inte = np.mean(group["Pint"][0])
    mini = np.mean(group["Pmin"][0])

    # Defining hyperparameters
    n = 50  # number of particles
    params = {"wmax" : 0.9, "wmin" : 0.4, "c1" : 2 , "c2" : 2}
    max_iteration = 100

    x = np.array([inte, mini])

    def func(X,P_sup = maxi):
        return np.square(P_sup - X[0])+np.square(P_sup - X[1])

    nVar= 2  # number of variables to optimize
    VarMin = 0  # lower bound of variables , you can use np.array() for different variables
    VarMax = 255   # upper bound of variables, you can use np.array() for different variables

    gbest = pso(func, max_iter=max_iteration, num_particles = n, dim = 2, vmin = VarMin, vmax = VarMax, params = params)

    #gamma correction for inferior color channels
    mean_colors = gbest['position']
    gamma = np.log(mean_colors/255)/np.log(x/255)

    group["Pint"][0] = np.array(255*np.power(group["Pint"][0]/255 , gamma[0]))
    group["Pmin"][0] = np.array(255*np.power(group["Pmin"][0]/255 , gamma[1]))


    pso_res = image(group.values())

    return pso_res

In [ ]:
def unsharp_masking(img):

    alpha = 0.2
    beta = 1 -alpha
    img_blur = cv.GaussianBlur(img, (1,1),sigmaX=1)
    unsharp_img = cv.addWeighted(img, alpha, img_blur, beta, 0.0)

    return unsharp_img


In [ ]:
def NUCE(img):
    neu_img = neutralize_image(img)

    #Dual-intensity images fusion based on
    #average of mean and median values
    img1, img2 = Stretching(neu_img)

    dual_img = enhanced_image(img1, img2)

    #Swarm-intelligence based mean equalization
    pso_res = pso_image(dual_img)

    #Unsharp masking (Gaussian blur)
    nuce_img = unsharp_masking(pso_res)

    return nuce_img

In [ ]:
dir_path = '/content/drive/MyDrive/UGP_EE492A/PSO_results/images/'
output_path =  '/content/drive/MyDrive/UGP_EE492A/PSO_results/results/'

In [ ]:
original_images =[]
NUCE_images =[]

In [ ]:
for im in os.listdir(dir_path):

  img = cv.imread(dir_path+im,1)
  original_images.append(img)

  nuce_img = NUCE(img)
  NUCE_images.append(nuce_img)

  # Save the output image to the output folder
  output_image_path = os.path.join(output_path, im)
  # cv.imwrite(output_image_path, nuce_img)

  # cv.imwrite("./results/"+im.split('/')[-1], nuce_img)
